In [12]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import Input, Flatten, Dense
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import os

In [13]:
# Define image size and directories
IMAGE_SIZE = [299, 299]
train_directory = r"C:\Users\castr\OneDrive\Desktop\img.class\melon_dataset\train"
test_directory = r"C:\Users\castr\OneDrive\Desktop\img.class\melon_dataset\test"
val_directory = r"C:\Users\castr\OneDrive\Desktop\img.class\melon_dataset\val"

In [14]:
# Add preprocessing layer to the front of InceptionV3
inception = InceptionV3(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [15]:
# Don't train existing weights
for layer in inception.layers:
    layer.trainable = False

In [16]:
# Get the number of classes
num_classes = len(os.listdir(train_directory))

In [17]:
# Build the model
x = Flatten()(inception.output)
prediction = Dense(num_classes, activation='softmax')(x)
model = Model(inputs=inception.input, outputs=prediction)

In [18]:
# Compile the model
model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(learning_rate=0.001),
    metrics=['accuracy']
)

In [19]:
# Data augmentation for training set
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

In [20]:

# Data augmentation for testing set (only rescaling)
test_datagen = ImageDataGenerator(rescale=1./255)

In [21]:
# Load training and testing sets with data augmentation
training_set = train_datagen.flow_from_directory(
    train_directory,
    target_size=(299, 299),
    batch_size=32,
    class_mode='categorical'
)

test_set = test_datagen.flow_from_directory(
    test_directory,
    target_size=(299, 299),
    batch_size=32,
    class_mode='categorical'
)

Found 1920 images belonging to 10 classes.
Found 880 images belonging to 10 classes.


In [25]:
# Train the model
history = model.fit(
    training_set,
    validation_data=test_set,
    epochs=10,
    steps_per_epoch=50,  # Adjust this value
    validation_steps=len(test_set) * 2
)

Epoch 1/10
50/50 [==============================] - 207s 4s/step - loss: 1.9288 - accuracy: 0.8481 - val_loss: 1.1244 - val_accuracy: 0.8943
Epoch 2/10
50/50 [==============================] - 136s 3s/step - loss: 1.3073 - accuracy: 0.8881
Epoch 3/10
50/50 [==============================] - 164s 3s/step - loss: 0.8239 - accuracy: 0.9187
Epoch 4/10
50/50 [==============================] - 231s 5s/step - loss: 1.0232 - accuracy: 0.9194
Epoch 5/10
50/50 [==============================] - 206s 4s/step - loss: 0.9001 - accuracy: 0.9388
Epoch 6/10
50/50 [==============================] - 184s 4s/step - loss: 0.4001 - accuracy: 0.9600
Epoch 7/10
50/50 [==============================] - 216s 4s/step - loss: 0.5309 - accuracy: 0.9581
Epoch 8/10
50/50 [==============================] - 226s 4s/step - loss: 0.5801 - accuracy: 0.9506
Epoch 9/10
50/50 [==============================] - 205s 4s/step - loss: 0.5480 - accuracy: 0.9550
Epoch 10/10
50/50 [==============================] - 208s 4s/step -

In [ ]:
# Save the model
model.save('melon_species_inceptionv3.h5')